In [1]:
import numpy as np
from numpy import genfromtxt
import math

In [2]:
class LoadDataset():
    
    def __init__(self):
        my_dict = {'s': 1, 'b': -1}
        self.y = np.loadtxt('../resources/train.csv', delimiter=",",dtype=np.str_,skiprows=1,usecols=1) 
        self.y = np.array([my_dict[i] for i in self.y])
        self.y = self.y.reshape(self.y.shape[0],1)
        self.x = np.loadtxt('../resources/train.csv', delimiter=",",skiprows=1,usecols=range(2,32))
        self.ids = np.loadtxt('../resources/train.csv', delimiter=",",skiprows=1,usecols=0)
        self.ids = self.ids.reshape(self.ids.shape[0],1)
        #Standardize Data Manually for each separate group of data after Distribution manipulation
        #self.x = (self.x - np.mean(self.x, axis=0))/np.std(self.x, axis=0)
        self.n_samples = self.x.shape[0]
            
    def __getitem__(self,i):
        return self.x[i] , self.y[i] , self.ids[i]
    
    def __len__(self):
        return self.n_samples
    
    def get_data(self):
        return self.x , self.y ,self.ids

In [3]:
train_data = LoadDataset()
tx , y ,ids = train_data.get_data()

In [4]:
def separate_data(tx,y,ids):
    
    array_1 = np.where( (tx[:,22]!=0))[0]
    tx_1 = np.delete(tx, array_1,0)
    y_1 = np.delete(y, array_1,0)
    ids_1 = np.delete(ids, array_1,0)
    
    array_2 = np.where( (tx[:,22]!=1))[0]
    tx_2 = np.delete(tx, array_2,0)
    y_2 = np.delete(y, array_2,0)
    ids_2 = np.delete(ids, array_2,0)
    
    array_3 = np.nonzero(np.logical_or((tx[:,22]==0), (tx[:,22]==1)))[0]
    tx_3 = np.delete(tx, array_3, 0)
    y_3 = np.delete(y, array_3, 0)
    ids_3 = np.delete(ids, array_3, 0)
    
    return (tx_1,y_1,ids_1) , (tx_2,y_2,ids_2) , (tx_3,y_3,ids_3)

In [5]:
def adapt_features(data_n):
    
    tx_n = data_n[0]
    to_delete = np.zeros(tx_n.shape[0])-999
    indexes = [] 
    for i in range(tx_n.shape[1]):
        #if(sum(tx_n[:,i]-to_delete)==0 or sum(tx_n[:,i])==0 ):
            #indexes.append(i)
        # Find unique values in column along with their length 
        # if len is == 1 then it contains same values i.e -999 so features to drop or zeros
        if len(np.unique(tx_n[:,i])) == 1:  
            indexes.append(i)
    
    tx_n =np.delete(tx_n,indexes,1)
    
    return (tx_n,data_n[1],data_n[2])

In [6]:
data_1,data_2,data_3 = separate_data(tx,y,ids)

In [7]:
data_1 = adapt_features(data_1)
data_2 = adapt_features(data_2)
data_3 = adapt_features(data_3)

In [8]:
def add_median(data_n):
    
    tx_n = data_n[0]
    for i in range(tx_n.shape[1]):
        column = tx_n[:,i]
        m = np.median(column[column!=-999])
        column[column == -999] = m
        tx_n[:,i]=column
        
    return (tx_n,data_n[1],data_n[2])

In [9]:
data_1 = add_median(data_1)
data_2 = add_median(data_2)
data_3 = add_median(data_3)

In [10]:
def dimensionality_reduction_corr(data_n):
    tx_n = data_n[0]
    corr = np.corrcoef(tx_n.T)
    pairs = np.argwhere(np.triu(np.isclose(corr,1,rtol=2e-01),1))
    tx_n = np.delete(tx_n,pairs[:,1],axis=1)
    return (tx_n,data_n[1],data_n[2])

In [11]:
data_1=dimensionality_reduction_corr(data_1)
data_2=dimensionality_reduction_corr(data_2)
data_3=dimensionality_reduction_corr(data_3)

In [73]:
def dimensionality_reduction_PCA(data_n,k):
    # Compare performance with dimen reduc correlation first
    tx_n = data_n[0]
    #standardize the data 
    tx_n = (tx_n - np.mean(tx_n, axis=0))/np.std(tx_n, axis=0)
    #compute covariance matrix 
    cov = np.cov(tx_n.T)
    #find eighenvalues and eighenvector
    ei_val, ei_vect = np.linalg.eig(cov)
    # Make list of tuples and sort in decreasing eighenvalue order 
    ei_tuple = [(np.abs(ei_val[i]), ei_vect[:, i]) for i in range(len(ei_val))]
    ei_tuple.sort(key = lambda k: k[0], reverse = True)
    #Keep the eighenvectors corresponding to the K largest eighenvalues (TRADE-OFF TO BE MADE)
    ei_tuple_largest = ei_tuple[:k]
    #Construct a projection matrix W from the these eigenvectors
    w = np.stack([vec for val, vec in ei_tuple_largest], axis=1)
    #Transform X using the projection matrix W to obtain the new k-dimensional feature subspace
    #TODO 
    return ei_tuple, ei_tuple_largest,w
    #return data_n

In [74]:
#Test
ei_tuple,ei_tuple_largest,w = dimensionality_reduction_PCA(data_1,5)

In [75]:
#Test
[x[1] for x in ei_tuple_largest][0].shape[0]

16

In [76]:
#Test
data_1[0].shape[1]

16

In [77]:
#Test
w.shape

(16, 5)

In [492]:
def scale_transform(data_n):
    tx_n = data_n[0]
    # Scaling Data to apply Log function
    dividor = np.max(tx_n,axis=0)-np.min(tx_n,axis=0)   
    tx_n = (tx_n - np.min(tx_n,axis=0))/dividor
    # Transforming Data
    tx_n = np.log(1+tx_n)
    # Normalizing Data
    centered_data = tx_n - np.mean(tx_n, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    return (std_data,data_n[1],data_n[2])

In [493]:
data_1 = scale_transform(data_1)
data_2 = scale_transform(data_2)
data_3 = scale_transform(data_3)

In [3]:
#Put it all together 
def process_data() : 
    #load the initial training set 
    train_data = LoadDataset()
    #extract features, predictions and ids
    tx , y ,ids = train_data.get_data()
    #separate data into 3 categories according to the PRI_JEST_NUM 
    data_1,data_2,data_3 = separate_data(tx,y,ids)
    #remove column that contain only 1 data value (-999, 0 or 1 in our case)
    data_1 = adapt_features(data_1)
    data_2 = adapt_features(data_2)
    data_3 = adapt_features(data_3)
    #replace -999 coefficients with the mediane of the column (-999 coefficients discared)
    data_1 = add_median(data_1)
    data_2 = add_median(data_2)
    data_3 = add_median(data_3)
    #dimentionality reduction : remove column that are strongly correlated
    data_1=dimensionality_reduction_corr(data_1)
    data_2=dimensionality_reduction_corr(data_2)
    data_3=dimensionality_reduction_corr(data_3)
    #TODO : add scaling - features explansion - log transformation 
    return (data_1[0],data_1[1],data_1[2]), (data_2[0],data_2[1],data_2[2]), (data_3[0],data_3[1],data_3[2])